In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
import pdir as pr
import pandas as pd
import os
from tqdm import tqdm, tnrange, tqdm_notebook

DF = pd.DataFrame
arr = np.array

# 读取数据

In [2]:
dirPath = "data preprocessed\\unnormalized"

trainSet_origin = np.loadtxt(dirPath + '\\train.csv', delimiter=",")
validateSet_origin = np.loadtxt(dirPath + '\\validate.csv', delimiter=",")
testSet = np.loadtxt(dirPath + '\\test.csv', delimiter=",")

trainSet_origin.shape, validateSet_origin.shape, testSet.shape

((33600, 18), (14400, 18), (12000, 17))

12-22：

决定做个尝试，直接用训练集和验证集一起训练KNN。参数使用step2中调出来的最优K值。【暂还没有理论依据证明此法有无用处】

12-23：

【测试结果】：在测试集上的F1值比step2的结果高了2%，所以还是有影响的。

那么问题又来了，step1的最优值，也即是k=1现在还可以适用于整个训练集的吗？

所以今天尝试地把k=2-5的结果都跑一便试试。看明天结果如何。

In [3]:
trainSet_merge = np.row_stack([trainSet_origin, validateSet_origin])
trainSet_merge.shape

(48000, 18)

划分数据集特征和标签：

In [4]:
trainSet, trainSetLabel = trainSet_merge[:, :-1], trainSet_merge[:, -1]
trainSet.shape

(48000, 17)

# 实现评测指标

In [5]:
def calcF1(predictLabel, realLabel, printRes=False):
    '''给定算法得到的分类结果和实际的分类结果，计算评测指标
    注意：标签为1则为正标签，标签不为1则为负标签'''
    def safeDivide(a, b):
        '''子函数：当分母为0时返回除式结果为 -1'''
        return a/b if b != 0 else 0
    
    TP = Counter(np.logical_and(predictLabel==1, realLabel==1))[True]
    FN = Counter(np.logical_and(predictLabel!=1, realLabel==1))[True]
    FP = Counter(np.logical_and(predictLabel==1, realLabel!=1))[True]
    if(printRes):
        print("TP FN FP", TP, FN, FP)
    F1 = safeDivide(2*TP, 2*TP+FN+FP)
    return F1

print('test 1: normal one')
calcF1(arr([1, 1, 0, 0]), arr([1, 0, 1, 0]), True)
print('\ntest 2: may have zero division error')
calcF1(arr([0, 0, 1, 0]), arr([1, 0, 1, 0]), True)

test 1: normal one
TP FN FP 1 1 1


0.5


test 2: may have zero division error
TP FN FP 1 1 0


0.6666666666666666

# KNN

In [6]:
def knn_classify(dataSet, labels, k, inputVector):
    '''使用knn对输入的向量进行分类，使用欧式距离'''
    #得到输入向量与数据集的差值的平方
    diffMat = (dataSet - inputVector)**2
    #计算每一行上元素的和并开方，得到距离向量
    distances = np.sqrt(diffMat.sum(axis=1))
    #print("输入向量与训练集每个向量的距离依次为：",list(distances))
    #得到 K个近邻的下标
    kNeighborsIndex  = distances.argpartition(k-1)[0:k]
    #返回分类结果
    return Counter(labels[kNeighborsIndex]).most_common(1)[0][0]

##############测试程序#######################
knn_classify(arr([[1,1,0,0],
              [0,1,1,1],
              [1,0,0,1]]), arr([1, 0, 1]), 1, arr([0,1,1,0]))
##############测试程序#######################

0

# 调参

所调参数只有K。使用step2的最优参数1.

In [7]:
def run_knn_classify(dataSet, k):
    ansLabel = []
    for i in tnrange(len(dataSet), leave=False):
        ansLabel.append(knn_classify(trainSet, trainSetLabel, k, dataSet[i]))
    return arr(ansLabel)

def getTestF1(k):
    ansLabel = run_knn_classify(validateSet, k)
    return calcF1(ansLabel, validateSetLabel)

# 应用算法到测试集

In [8]:
ansLabel = run_knn_classify(testSet, k=2)
np.savetxt('.\\rank\\47_v1.csv', ansLabel, fmt="%d", delimiter="\n")

In [9]:
ansLabel = run_knn_classify(testSet, k=3)
np.savetxt('.\\rank\\47_v2.csv', ansLabel, fmt="%d", delimiter="\n")

In [10]:
ansLabel = run_knn_classify(testSet, k=4)
np.savetxt('.\\rank\\47_v3.csv', ansLabel, fmt="%d", delimiter="\n")

In [11]:
ansLabel = run_knn_classify(testSet, k=5)
np.savetxt('.\\rank\\47_v4.csv', ansLabel, fmt="%d", delimiter="\n")

In [12]:
Counter(ansLabel)
one = Counter(ansLabel)[1.0]
zero = Counter(ansLabel)[0.0]
one/(one + zero)

Counter({0.0: 5990, 1.0: 6010})

0.5008333333333334